# Dual Numbers and Algorithmic Differentiation basics
Dual numbers are similar to complex numbers but with a second element coefficient  ${\epsilon}$ that squares to zero (as opposed to $-1$ for complex numbers).


https://en.wikipedia.org/wiki/Automatic_differentiation

In [20]:
from math import pi, sin, cos, sqrt
print(37.5*sin(pi/6.0))

18.749999999999996


In [26]:
##### start dual number class #####

class Dual:
    """
    Python dual numbers class 
    """
    def __init__(self, real: float, dual: float):
        self.a = float(real)
        self.b = float(dual)
    
    def __repr__(self):
        return str(self.a) + "+" + str(self.b) +"*epsilon"
    
    def __str__(self):
        return str(self.a) + "+" + str(self.b) +"\xce\xb5"
   
    def __nonzero__(self):
        return(self.a == 0 and self.b == 0)   
    
    def __add__(self, other):
        if isinstance(other, (float, int)):
            return Dual(other+self.a, self.b)
        return Dual(other.a+self.a, other.b+self.b)
    
    def __sub__(self, other):
        if isinstance(other, (float, int)):
            return Dual(self.a-other, self.b)
        return Dual(other.a-self.a, other.b-self.b)
    
    def __mul__(self, other):
        if isinstance(other, (float, int)):
            return Dual(other*self.a,other*self.b)
        return Dual(self.a*other.a, self.a*other.b + self.b*other.a)
    
    def __truediv__(self, other):
        if isinstance(other, (float, int)):
            return Dual(self.a/other, self.b/other)
        return Dual(self.a/other.a, (self.b*other.a - self.a*other.b)/(other.a*other.a)) 
    
    def __floordiv__(self, other):
        if isinstance(other, (float, int)):
            return Dual(self.a/other, self.b/other)
        return Dual(self.a/other.a, (self.b*other.a - self.a*other.b)/(other.a*other.a)) 
    
    def __pow__(self, other):
        assert isinstance(other, int), "only int powers are supported"
        if other == 1:
            return self
        elif other == 2:
            return Dual(self.a**2, 2*self.a*self.b)
        elif other > 1:
            return self * self**(other-1)
        else:
            return (self**(other+1))/self
    
    def __radd__(self, other):
        if isinstance(other, (float, int)):
            return Dual(other+self.a, self.b)
        return Dual(other.a+self.a, other.b+self.b)
    
    def __rsub__(self, other):
        if isinstance(other, (float, int)):
            return Dual(-self.a+other, -self.b)
        return Dual(other.a-self.a, other.b-self.b)
    
    def __rmul__(self, other):
        if isinstance(other, (float, int)):
            return Dual(other*self.a,other*self.b)
        return Dual(self.a*other.a, self.a*other.b + self.b*other.a)
    
    def __neg__(self):
        return Dual(-self.a, -self.b)
    
    def __pos__(self):
        return self
    
    def __abs__(self):
        return abs(self.a)
    

epsilon = Dual(0, 1) 

In [27]:
x1=Dual(49,0)
x2=Dual(-7,2)
x2/x1

-0.14285714285714285+0.04081632653061224*epsilon

# Example 1  $f(x)=x^3$; $\frac{\partial {f}}{\partial {x}} = 3x^2$
By the fundamental properties of [dual numbers](https://en.wikipedia.org/wiki/Dual_number):  

$$f(a + b\epsilon) = f(a) + bf'(a)\epsilon$$where $f'(x)$ is the first derivative of $f(x)$


With duals, we are at liberty to set the second element $b$ to $1$, so:
$$f(a + b\epsilon) = f(a) + f'(a)\epsilon$$


In python, the dualNum class above implements dual numbers and operations on them. The second element, in a manner completely analogous to complex numbers where the second element is imaginary, is implicitly multiplied by $\epsilon$.

The below snippet shows how [Algorithmic Differentiation](https://en.wikipedia.org/wiki/Algorithmic_differentiation) is realised without re-structuring/re-factoring or other instrumentation, other than substitution of scalar real number variables with dualNum variables.

In [28]:
#Calculate f(x) and f'(x) where f(x)=x^3
#Set x=5, with second element = 1 as discussed above
x=Dual(5,1)
y=x**3
print("For x={}: f(x) = {}, f'(x) = {}".format(x.a, y.a, y.b))

For x=5.0: f(x) = 125.0, f'(x) = 75.0


# Functions of more than one variable and "sweeps"
The case for functions $f(x_1, x_2, ...)$ of more than one variable complicates things slightly. The basic approach is as follows:
- Ensure all input variables ($x_1, x_2$, etc.) are dual numbers
- To recover the partial derivative $\frac{\partial {f}}{\partial {x_i}}$, set "b" element (the second, or $\epsilon$ element) of $x_i = 1$
- Set "b" elements of all other variables to 0 ( $x_j$ where $j \neq i$ are effectively real-only numbers)
- Evaluate result
- Repeat this for each variable
- AD Jargon : each iteration of above is called a "sweep"

# AD Example for Functions of more than one variable $f(x_1, x_2, ...)$
# Example 2 : $f(x_1,x_2)=\frac{x_1^3}{x_2}$ 
We know that (from basic differential calculus - no AD yet!):
- $\frac{\partial {f}}{\partial {x_1}} = \frac{3x_1^2}{x_2}$
- $\frac{\partial {f}}{\partial {x_2}} = -\frac{x_1^3}{x_2^2}$

In [29]:
# Calculate f(x1,x2) and f'(x1), f'(x2) where f(x1, x2)=x1^3/x2; Use "sweeps" for partials wrt inputs
def f(x1, x2): return (x1**3)/x2
    
#Sweep 1 get f(x1,x2) and f'(x1); set x1.b=1 and any other xn.b=0
x1=Dual(5.0,1)
x2=Dual(2.0,0)
y=f(x1,x2)
print("For x1, x2 = {}, {}: f(x) = {}, f'(x1) = {}".format(x1.a, x2.a, y.a, y.b))

#Sweep 2 get f(x1,x2) and f'(x2); set x2.b=1 and any other xn.b=0
x1=Dual(5.0,0)
x2=Dual(2.0,1)
y=f(x1,x2)
print("For x1, x2 = {}, {}: f(x) = {}, f'(x2) = {}".format(x1.a, x2.a, y.a, y.b))

For x1, x2 = 5.0, 2.0: f(x) = 62.5, f'(x1) = 37.5
For x1, x2 = 5.0, 2.0: f(x) = 62.5, f'(x2) = -31.25


In [30]:
from math import pi, sin, cos
print(37.5*sin(pi/6.0))

18.749999999999996


# Adapting AD by "wrapping" functions
In pure Python, everything is easy (almost!) - we can introduce dual numbers and retain existing code, pretty much without significant change.

When we have a function which is implemented in another language, or when we have only a compiled version, we can "wrap" the function, so as to create an AD-enabled version of the  function which takes, and returns, dual numbers.

In our next example, we have a composite function in which we call a function in the "math" module, which is normally supplied as a pre-compiled dynamic link library (DLL - on Windows).

# Example 3
Here we take the previous example and introduce a dependency on a function ($sin(x_3)$), which is supported by a compiled library (e.g. DLL) for which source code is not immediately available. Since the functional form of such functions is often well understood, and analytical derivatives are well known or can be obtained, we therefore can "wrap" the objective function so it works with dual number AD implementation. 

For $f(x_1,x_2,x_3)=\frac{x_1^3}{x_2}sin(x_3)$ we know that (from basic differential calculus - no AD yet!):
- $\frac{\partial {f}}{\partial {x_1}} = \frac{3x_1^2}{x_2}sin(x_3)$
- $\frac{\partial {f}}{\partial {x_2}} = -\frac{x_1^3}{x_2^2}sin(x_3)$
- $\frac{\partial {f}}{\partial {x_3}} = \frac{x_1^3}{x_2}cos(x_3)$

In fact this technique can be automated somewhat in that known functional forms can be identified and appropriate functions "wrapped", then automated edits can substitute calls to functions with the AD versions.

In [39]:
from math import pi, sin, cos
#Wrapped dual number version of sin 
def _AD_sin(x):
    #We know that d/dx of sin(x) is cos(x); only return "b" value if x.b=1!!
    return Dual(sin(x.a), cos(x.a)*x.b)

def f(x1, x2, x3):
    return ((x1**3)/x2)*_AD_sin(x3)
    
#Sweep 1 get f(x1,x2, x3) and f'(x1); set x1.b=1 and any other xn.b=0
x1=Dual(5.0,1)
x2=Dual(2.0,0)
x3=Dual(pi/6,0)

y=f(x1,x2, x3)
print("For x1, x2, x3 = {}, {}, {}: f(x) = {}, f'(x1) = {}".format(x1.a, x2.a, x3.a, y.a, y.b))

#Sweep 2 get f(x1,x2, x3) and f'(x2); set x2.b=1 and any other xn.b=0
x1=Dual(5.0,0)
x2=Dual(2.0,1)
x3=Dual(pi/6,0)

y=f(x1,x2, x3)
print("For x1, x2, x3 = {}, {}, {}: f(x) = {}, f'(x2) = {}".format(x1.a, x2.a, x3.a, y.a, y.b))

#Sweep 3 get f(x1,x2, x3) and f'(x3); set x3.b=1 and any other xn.b=0
x1=Dual(5.0,0)
x2=Dual(2.0,0)
x3=Dual(pi/6,1)

y=f(x1,x2, x3)
print("For x1, x2, x3 = {}, {}, {}: f(x) = {}, f'(x3) = {}".format(x1.a, x2.a, x3.a, y.a, y.b))

For x1, x2, x3 = 5.0, 2.0, 0.5235987755982988: f(x) = 31.249999999999996, f'(x1) = 18.749999999999996
For x1, x2, x3 = 5.0, 2.0, 0.5235987755982988: f(x) = 31.249999999999996, f'(x2) = -15.624999999999998
For x1, x2, x3 = 5.0, 2.0, 0.5235987755982988: f(x) = 31.249999999999996, f'(x3) = 54.12658773652742


In [32]:
from math import pi, sin, cos
#Wrapped dual number version of sin 
def _AD_sin(x):
    #We know that d/dx of sin(x) is cos(x).
    return Dual(sin(x.a), cos(x.a)*x.b)

def f(x1, x2, x3):
    return ((x1**3)/x2)*_AD_sin(x3**2)
    
#Sweep 1 get f(x1,x2, x3) and f'(x1); set x1.b=1 and any other xn.b=0
x1=Dual(5.0,1)
x2=Dual(2.0,0)
x3=Dual((pi/6)**0.5,0)

y=f(x1,x2, x3)
print("For x1, x2, x3 = {}, {}, {}: f(x) = {}, f_x1(x) = {}".format(x1.a, x2.a, x3.a, y.a, y.b))

#Sweep 2 get f(x1,x2, x3) and f'(x2); set x2.b=1 and any other xn.b=0
x1=Dual(5.0,0)
x2=Dual(2.0,1)
x3=Dual((pi/6)**0.5,0)

y=f(x1,x2, x3)
print("For x1, x2, x3 = {}, {}, {}: f(x) = {}, f_x2(x) = {}".format(x1.a, x2.a, x3.a, y.a, y.b))

#Sweep 3 get f(x1,x2, x3) and f'(x3); set x3.b=1 and any other xn.b=0
x1=Dual(5.0,0)
x2=Dual(2.0,0)
x3=Dual((pi/6)**0.5,1)

y=f(x1,x2, x3)
print("For x1, x2, x3 = {}, {}, {}: f(x) = {}, f_x3(x) = {}".format(x1.a, x2.a, x3.a, y.a, y.b))

For x1, x2, x3 = 5.0, 2.0, 0.7236012545582676: f(x) = 31.249999999999996, f_x1(x) = 18.749999999999996
For x1, x2, x3 = 5.0, 2.0, 0.7236012545582676: f(x) = 31.249999999999996, f_x2(x) = -15.624999999999998
For x1, x2, x3 = 5.0, 2.0, 0.7236012545582676: f(x) = 31.249999999999996, f_x3(x) = 78.33213358221877


In [33]:
-0.5*31.25

-15.625

In [34]:
2 * 5**3 * (pi/6)**0.5 * cos(pi/6) / 2

78.33213358221877

In [35]:
1/2/3

0.16666666666666666

In [36]:
-x1

-5.0+-0.0*epsilon

In [37]:
abs(x1)

5.0

In [38]:
abs(x1)

5.0